<a href="https://colab.research.google.com/github/niz11/Bachelor_Thesis_Colab_Files/blob/main/5_frames_composite_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/dataset_5_frame_seq_faces/X_5.npy')
y = np.load('drive/My Drive/dataset_5_frame_seq_faces/Y_label_5.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
import keras
from keras.layers import Input, Flatten, Dense,LSTM,TimeDistributed,RepeatVector
from keras.models import Model,Sequential
from keras.utils import plot_model


def create_model(encoding_length=64,sequence_length=10):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(sequence_length, encoding_length))
  encoder = LSTM(encoding_length, return_sequences=True)(encoder_inputs)
  encoder_outputs, state_h, state_c  = LSTM(encoding_length, return_state=True, name='encoder')(encoder)
  encoder_states = [state_h, state_c]
  clssifier  = Dense(7, activation='softmax')(encoder_outputs)

  model = Model(inputs=encoder_inputs, outputs=[clssifier])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  plot_model(model, show_shapes=True, to_file='lstm_autoencoder.png')
  model.summary()
  return model

model = create_model(128, 5)
model.load_weights("drive/My Drive/trained_models/composite_paper_5/encoderComposite_5.h5",by_name=True)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 128)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 5, 128)            131584    
_________________________________________________________________
encoder (LSTM)               [(None, 128), (None, 128) 131584    
_________________________________________________________________
dense (Dense)                (None, 7)                 903       
Total params: 264,071
Trainable params: 264,071
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.05)

Epoch 1/10
60/60 [==============================] - 1s 25ms/step - loss: 1.5537 - accuracy: 0.4225 - val_loss: 1.2809 - val_accuracy: 0.6400
Epoch 2/10
60/60 [==============================] - 0s 8ms/step - loss: 0.8282 - accuracy: 0.7626 - val_loss: 0.8705 - val_accuracy: 0.7700
Epoch 3/10
60/60 [==============================] - 0s 8ms/step - loss: 0.4232 - accuracy: 0.8932 - val_loss: 0.6034 - val_accuracy: 0.8700
Epoch 4/10
60/60 [==============================] - 0s 8ms/step - loss: 0.1994 - accuracy: 0.9593 - val_loss: 0.5628 - val_accuracy: 0.8600
Epoch 5/10
60/60 [==============================] - 0s 8ms/step - loss: 0.1172 - accuracy: 0.9783 - val_loss: 0.4270 - val_accuracy: 0.9300
Epoch 6/10
60/60 [==============================] - 0s 8ms/step - loss: 0.0694 - accuracy: 0.9878 - val_loss: 0.4965 - val_accuracy: 0.9100
Epoch 7/10
60/60 [==============================] - 0s 8ms/step - loss: 0.0493 - accuracy: 0.9915 - val_loss: 0.4423 - val_accuracy: 0.9100
Epoch 8/10
60/60 [=

In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
  else:
    print(f'Wrong classification, truth: {getFaceExpressionFromIndex(truth)}')
    print(f'Wrong classification, prediction: {getFaceExpressionFromIndex(prediction)}')
    print("---------------------------------------------------------------------------")
print(f'number of samples: {len(test_predictions)}')
print(f'correct: {correct}')
print(f'Accuracy of predicitons: {correct / len(test_predictions)}')

Wrong classification, truth: anger
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: sad
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: anger
---------------------------------------------------------------------------
Wrong classification, truth: surprise
Wrong classification, prediction: anger
---------------------------------------------------------------------------
Wrong classification, truth: none
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: anger
Wrong classification, prediction: surprise
---------------------

In [ ]:
# Here and below is the network to classify the encodings

In [ ]:
from keras.models import load_model
from keras.models import model_from_json
import json

# load json and create model
json_file = open('drive/My Drive/trained_models/composite_paper_5/encoderComposite_5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
encoder = model_from_json(loaded_model_json)
# load weights into new model
encoder.load_weights("drive/My Drive/trained_models/composite_paper_5/encoderComposite_5.h5")
print("Loaded model from disk")


Loaded model from disk


In [ ]:
# Check here: better divinding to test/train set
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/dataset_5_frame_seq_faces/X_5.npy')
y = np.load('drive/My Drive/dataset_5_frame_seq_faces/Y_label_5.npy')

predictions = encoder.predict(X)


X_train, X_test, y_train, y_test = train_test_split(predictions, y, test_size = 0.1, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


def classifier():
  model = Sequential()
  model.add(Dense(64, input_dim=128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = classifier()
model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.1)

Epoch 1/50
56/56 [==============================] - 0s 6ms/step - loss: 1.8153 - accuracy: 0.2730 - val_loss: 1.8007 - val_accuracy: 0.2900
Epoch 2/50
56/56 [==============================] - 0s 4ms/step - loss: 1.6158 - accuracy: 0.4042 - val_loss: 1.6645 - val_accuracy: 0.4050
Epoch 3/50
56/56 [==============================] - 0s 4ms/step - loss: 1.4493 - accuracy: 0.5003 - val_loss: 1.5150 - val_accuracy: 0.4550
Epoch 4/50
56/56 [==============================] - 0s 4ms/step - loss: 1.2973 - accuracy: 0.5572 - val_loss: 1.4202 - val_accuracy: 0.5350
Epoch 5/50
56/56 [==============================] - 0s 4ms/step - loss: 1.1477 - accuracy: 0.6343 - val_loss: 1.2808 - val_accuracy: 0.5750
Epoch 6/50
56/56 [==============================] - 0s 4ms/step - loss: 1.0232 - accuracy: 0.6767 - val_loss: 1.2103 - val_accuracy: 0.6150
Epoch 7/50
56/56 [==============================] - 0s 4ms/step - loss: 0.9159 - accuracy: 0.7097 - val_loss: 1.1004 - val_accuracy: 0.6800
Epoch 8/50
56/56 [==

In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
  else:
    print(f'Wrong classification, truth: {getFaceExpressionFromIndex(truth)}')
    print(f'Wrong classification, prediction: {getFaceExpressionFromIndex(prediction)}')
    print("---------------------------------------------------------------------------")

print(f'Accuracy of predicitons: {correct / len(test_predictions)}')
print(f'Got correct: {correct}')
print(f'Got wrong: {len(test_predictions) - correct}')

Wrong classification, truth: anger
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: sad
Wrong classification, prediction: disgust
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: surprise
---------------------------------------------------------------------------
Wrong classification, truth: anger
Wrong classification, prediction: surprise
---------------------------------------------------------------------------
Wrong classification, truth: surprise
Wrong classification, prediction: anger
---------------------------------------------------------------------------
Wrong classification, truth: none
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: anger
Wrong classification, prediction: smile
--------------